In [1]:
import pandas as pd
import requests
from datetime import datetime
from IPython.display import Image, display
from base64 import b64encode, b64decode

In [2]:
df = pd.read_csv('eonet_data.csv')
df.head()

,_id,id,link,title,Category ID,Category Title,Magnitude Value,Magnitude Unit,Geometry Type,Source ID,Source URL,Date,Time,Longitude,Latitude
0,Row(oid='6506a0255457d4ab94009af1'),EONET_6425,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,"Kilauea Volcano, Hawaii",volcanoes,Volcanoes,NaN,NaN,Point,SIVolcano,https://volcano.si.edu/volcano.cfm?vn=332010,2023-09-10,00:00:00Z,-155.287,19.421
1,Row(oid='6506a0255457d4ab94009af2'),EONET_6423,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33A,seaLakeIce,Sea and Lake Ice,340.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-08,00:00:00Z,17.140,-69.380
2,Row(oid='6506a0255457d4ab94009af2'),EONET_6423,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33A,seaLakeIce,Sea and Lake Ice,340.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-15,00:00:00Z,14.370,-69.230
3,Row(oid='6506a0255457d4ab94009af3'),EONET_6424,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33B,seaLakeIce,Sea and Lake Ice,252.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-08,00:00:00Z,18.300,-69.640
4,Row(oid='6506a0255457d4ab94009af3'),EONET_6424,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33B,seaLakeIce,Sea and Lake Ice,252.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-15,00:00:00Z,16.600,-69.450


# Earth planet image by date

In [3]:
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [4]:
df.head()

,_id,id,link,title,Category ID,Category Title,Magnitude Value,Magnitude Unit,Geometry Type,Source ID,Source URL,Date,Time,Longitude,Latitude
0,Row(oid='6506a0255457d4ab94009af1'),EONET_6425,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,"Kilauea Volcano, Hawaii",volcanoes,Volcanoes,NaN,NaN,Point,SIVolcano,https://volcano.si.edu/volcano.cfm?vn=332010,2023-09-10,00:00:00Z,-155.287,19.421
1,Row(oid='6506a0255457d4ab94009af2'),EONET_6423,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33A,seaLakeIce,Sea and Lake Ice,340.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-08,00:00:00Z,17.140,-69.380
2,Row(oid='6506a0255457d4ab94009af2'),EONET_6423,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33A,seaLakeIce,Sea and Lake Ice,340.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-15,00:00:00Z,14.370,-69.230
3,Row(oid='6506a0255457d4ab94009af3'),EONET_6424,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33B,seaLakeIce,Sea and Lake Ice,252.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-08,00:00:00Z,18.300,-69.640
4,Row(oid='6506a0255457d4ab94009af3'),EONET_6424,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,Iceberg D33B,seaLakeIce,Sea and Lake Ice,252.0,NM^2,Point,NATICE,https://usicecenter.gov/pub/Iceberg_Tabular.csv,2023-09-15,00:00:00Z,16.600,-69.450


In [9]:
base_url = 'https://api.nasa.gov/EPIC/api/natural'
api_key = "fFiJYzeHnnjHtYfhpXFyKS8qzjCNv7Cg6ecL5I5h"  

# Iterate through the DataFrame rows
for index, row in df.iterrows():
    date = row['Date']
    request_url = f'{base_url}/{date}?api_key={api_key}'

    # Make the API request
    response = requests.get(request_url)
    
    if response.status_code == 200:
        # Parse the JSON response to get the image URL
        data = response.json()
        img_date = data[0]['date']
        formatted_date = datetime.strptime(img_date, '%Y-%m-%d %H:%M:%S').strftime('%Y/%m/%d')
        epic_url = 'https://epic.gsfc.nasa.gov/archive/natural/'
        img_name = data[0]['image']
        image_url = f'{epic_url}{formatted_date}/png/{img_name}.png'
        
        # display the image
        display(Image(url=image_url))
        
    break

# Get the images by categories titles

In [10]:
def generate_urls_from_column(df, column_name):
    unique_values = df[column_name].unique()
    base_url = "https://images-api.nasa.gov/search?q="  
    urls_dict = {category: base_url + category.replace(" ", "-") + "&media_type=image" for category in unique_values}
    return urls_dict

column_name = 'Category Title'
urls_dict = generate_urls_from_column(df, column_name)

In [11]:
urls_dict

{'Volcanoes': 'https://images-api.nasa.gov/search?q=Volcanoes&media_type=image',
 'Sea and Lake Ice': 'https://images-api.nasa.gov/search?q=Sea-and-Lake-Ice&media_type=image',
 'Severe Storms': 'https://images-api.nasa.gov/search?q=Severe-Storms&media_type=image',
 'Wildfires': 'https://images-api.nasa.gov/search?q=Wildfires&media_type=image'}

In [12]:
# Define a function to create a DataFrame with specific category, img_url, and img_content
def create_df_from_specific_category(urls_dict):
    # Initialize lists to store data
    image_urls = []
    image_contents = []
    categories = []

    try:
        
        for key, value in urls_dict.items():
            category = key
            response = requests.get(value)
            response.raise_for_status()  # Check for HTTP request errors
            data = response.json()
            all_urls = [item['links'][0]['href'] for item in data['collection']['items']]
            for i, url in enumerate(all_urls, start=1):
                img_response = requests.get(url)
                image_content = img_response.content
                
                # Append data to lists
                image_contents.append(image_content)
                image_urls.append(url)
                categories.append(key)
             
            
            
    except Exception as e:
        print(f"Error while fetching or processing data: {str(e)}")

    df_result = pd.DataFrame({'Category': categories, 'Image_URL': image_urls, 'Image_Content': image_contents})
    return df_result


img_df = create_df_from_specific_category(urls_dict)

In [18]:
# count the images amount in category
for category, url in urls_dict.items():
# for url in urls:
    try:
            response = requests.get(url)
            response.raise_for_status()  # Check for HTTP request errors

            data = response.json()
            image_count = data['collection']['metadata']['total_hits']
            print(category, image_count)
    except Exception as e:
            print(f"Error while counting images for {url}: {str(e)}")         

Volcanoes 1169
Sea and Lake Ice 33
Severe Storms 392
Wildfires 300


# Insert Images to Cassandra DB

In [ ]:
# open ubuntu -> sudo service cassandra start

In [19]:
from cassandra.cluster import Cluster
import uuid

In [20]:
uuid.uuid4()

UUID('f6b3da32-0215-4184-8645-7137d0f05531')

In [21]:
cluster = Cluster(["127.0.0.1"],port=9042)

In [22]:
session = cluster.connect()

In [88]:
# create_keyspace = """
#     CREATE KEYSPACE nasa_meteorites
#     WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
# """

# session.execute(create_keyspace)

AlreadyExists: Keyspace 'nasa_meteorites' already exists

In [23]:
session.execute("USE nasa_meteorites")

In [113]:
# session.execute(""" DROP TABLE meteorites""")

In [90]:
# session.execute(""" 
# CREATE TABLE nasa_images (
#     id UUID PRIMARY KEY,
#     Category text,
#     Image_URL text,
#     Image_Content blob)
#     """)

In [95]:
# Define a prepared INSERT statement
insert_statement = session.prepare("""
    INSERT INTO nasa_images (id, Category, Image_URL, Image_Content)
    VALUES (?, ?, ?, ?)
""")

# Iterate over DataFrame rows and insert data into Cassandra
for _, row in img_df.iterrows():
    id = uuid.uuid4()  # Generate a UUID for the id field
    session.execute(insert_statement, (id, row['Category'], row['Image_URL'], row['Image_Content']))

In [24]:
result = session.execute("""SELECT id, Category, Image_URL FROM nasa_images LIMIT 10""")
for row in result:
    print(row.id, row.category, row.image_url)

bf2550b2-d26b-4609-9c73-b82e7d5d964d Wildfires https://images-assets.nasa.gov/image/PIA22554/PIA22554~thumb.jpg
e7f8693f-c2a7-4958-ac08-ab7c4bd96b72 Wildfires https://images-assets.nasa.gov/image/ED06-0208-3/ED06-0208-3~thumb.jpg
def19cc8-d86f-457f-b0d1-7bebc6030f14 Severe Storms https://images-assets.nasa.gov/image/KSC01pp0795/KSC01pp0795~thumb.jpg
2c86e5d1-3747-4a9f-ba53-178984f337cc Wildfires https://images-assets.nasa.gov/image/PIA20886/PIA20886~thumb.jpg
b50de13f-f0c0-49a6-9b6d-b20d0c699980 Volcanoes https://images-assets.nasa.gov/image/PIA01267/PIA01267~thumb.jpg
fcd03b29-606f-4fb1-b9e8-f7bc2e9ea0aa Volcanoes https://images-assets.nasa.gov/image/PIA02803/PIA02803~thumb.jpg
4892c0a9-bf1d-4c17-ac23-5d2c743b46b2 Severe Storms https://images-assets.nasa.gov/image/KSC-20161119-PH_KLS02_0078/KSC-20161119-PH_KLS02_0078~thumb.jpg
b774f79f-c26e-429d-971c-24c07523ddd6 Sea and Lake Ice https://images-assets.nasa.gov/image/iss012e15918/iss012e15918~thumb.jpg
92d7d4ee-6562-4b9f-a9e3-de6ec5a48

# Display the image from Cassandra DB

In [26]:
from PIL import Image
import io
from IPython.display import display, HTML, clear_output

In [25]:
img_id = 'bf2550b2-d26b-4609-9c73-b82e7d5d964d'

query = f"""SELECT Image_Content FROM nasa_images WHERE id = {img_id} ALLOW FILTERING"""

In [27]:
result = session.execute(query)
image_data = result.one()

In [28]:
image_bytes = image_data.image_content

In [29]:
# hex -> base64
s = image_bytes.hex()
b64 = b64encode(bytes.fromhex(s)).decode()
image_b64 =  b64

In [30]:
content_type = 'image/jpg'

In [31]:
image_html = f'<img src="data:{content_type};base64, {image_b64}" alt="Image">'
clear_output()
display(HTML(image_html))

In [40]:
# Close the Cassandra session and cluster when done
session.shutdown()
cluster.shutdown()